In [1]:
(require '[clojupyter.javascript.alpha :as cjp-js])
(require '[clojupyter.display :as display])
(require '[clojupyter.misc.helper :as helper])
(require '[clojure.data.json :as json])
(helper/add-dependencies '[org.clojure/data.csv "1.0.0"])
(require '[clojure.data.csv :as csv])
(helper/add-dependencies '[metasoarous/oz "1.5.6"])
(require '[oz.notebook.clojupyter :as oz])
(require '[clojure.java.io :as io])
(require '[clojure.pprint :as pp])
(helper/add-dependencies '[clojure.java-time "0.3.2"])
(require '[java-time :as t])
(require '[clojure.edn :as edn])

nil

We're going to use a new dataset here, to demonstrate how to deal with larger datasets. This is a subset of the of 311 service requests from [NYC Open Data](https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9). 

In [2]:
;; Python: complaints = pd.read_csv('../data/311-service-requests.csv')

;; read data in
(def raw-data
    (with-open [reader (io/reader "../data/311-service-requests.csv")]
      (doall
        (csv/read-csv reader))))

(defn blank->nil [s]
  (when-not (clojure.string/blank? s) s))

(defn csv-data->maps [csv-data]
  (map zipmap
       (->> (first csv-data) ;; First row is the header
            (map keyword) ;; Drop if you want string keys instead
            repeat)
       (->> (rest csv-data)
            (map #(map blank->nil %))))) ;; Drop if you want blank strings to stay

(def fixed-df (csv-data->maps raw-data))

#'user/fixed-df

In [6]:
(pp/print-table (map #(into {} (take 5 %)) (take 10 fixed-df)))


| :Road Ramp | :Resolution Action Updated Date | :Bridge Highway Name | :Park Facility Name | :School Number |
|------------+---------------------------------+----------------------+---------------------+----------------|
|            |          10/31/2013 02:35:17 AM |                      |         Unspecified |    Unspecified |
|            |                                 |                      |         Unspecified |    Unspecified |
|            |          10/31/2013 02:39:42 AM |                      |         Unspecified |    Unspecified |
|            |          10/31/2013 02:21:10 AM |                      |         Unspecified |    Unspecified |
|            |          10/31/2013 01:59:54 AM |                      |         Unspecified |    Unspecified |
|            |                                 |                      |         Unspecified |    Unspecified |
|            |          10/31/2013 01:59:51 AM |                      |         Unspecified |    Unspecified |


nil

# 2.2 Selecting columns and rows

To select a column, we index with the name of the column, like this:

In [7]:
(run! println (map (keyword "Complaint Type") (take 10 fixed-df)))

Noise - Street/Sidewalk
Illegal Parking
Noise - Commercial
Noise - Vehicle
Rodent
Noise - Commercial
Blocked Driveway
Noise - Commercial
Noise - Commercial
Noise - Commercial


nil

To get the first 5 rows of a dataframe, we can use a slice: `df[:5]`.

This is a great way to get a sense for what kind of information is in the dataframe -- take a minute to look at the contents and get a feel for this dataset.

In [62]:
(pp/pprint (take 5 fixed-df))

({:Road Ramp nil,
  :Resolution Action Updated Date "10/31/2013 02:35:17 AM",
  :Bridge Highway Name nil,
  :Park Facility Name "Unspecified",
  :School Number "Unspecified",
  :Park Borough "QUEENS",
  :Taxi Pick Up Location nil,
  :Address Type "ADDRESS",
  :Due Date "10/31/2013 10:08:41 AM",
  :Bridge Highway Segment nil,
  :School Phone Number "Unspecified",
  :Cross Street 1 "90 AVENUE",
  :Vehicle Type nil,
  :Bridge Highway Direction nil,
  :Complaint Type "Noise - Street/Sidewalk",
  :Y Coordinate (State Plane) "197389",
  :City "JAMAICA",
  :School Address "Unspecified",
  :Intersection Street 2 nil,
  :School State "Unspecified",
  :Agency "NYPD",
  :Unique Key "26589651",
  :School Code "Unspecified",
  :Intersection Street 1 nil,
  :School Zip "Unspecified",
  :Descriptor "Loud Talking",
  :Borough "QUEENS",
  :Street Name "169 STREET",
  :Incident Zip "11432",
  :Longitude "-73.79160395779721",
  :Agency Name "New York City Police Department",
  :Community Board "12 QUEENS

nil

We can combine these to get the first 5 rows of a column:

In [69]:
(run! println (->> fixed-df
                  (map (keyword "Complaint Type"))
                  (take 5)))

Noise - Street/Sidewalk
Illegal Parking
Noise - Commercial
Noise - Vehicle
Rodent


nil

and it doesn't matter which direction we do it in:

In [70]:
(run! println (->> fixed-df
                  (take 5)
                  (map (keyword "Complaint Type"))))

Noise - Street/Sidewalk
Illegal Parking
Noise - Commercial
Noise - Vehicle
Rodent


nil

# 2.3 Selecting multiple columns


What if we just want to know the complaint type and the borough, but not the rest of the information? Pandas makes it really easy to select a subset of the columns: just index with list of columns you want.

In [93]:
(->> fixed-df
    (take 5)
    (map (juxt (keyword "Complaint Type") (keyword "Borough"))))
(pp/pp)

(["Noise - Street/Sidewalk" "QUEENS"]
 ["Illegal Parking" "QUEENS"]
 ["Noise - Commercial" "MANHATTAN"]
 ["Noise - Vehicle" "MANHATTAN"]
 ["Rodent" "MANHATTAN"])


nil

In [94]:
;; or print as a table
(pp/print-table (->> fixed-df
                     (take 5)
                     (map #(select-keys % [(keyword "Complaint Type") (keyword "Borough")]))))


|         :Complaint Type |  :Borough |
|-------------------------+-----------|
| Noise - Street/Sidewalk |    QUEENS |
|         Illegal Parking |    QUEENS |
|      Noise - Commercial | MANHATTAN |
|         Noise - Vehicle | MANHATTAN |
|                  Rodent | MANHATTAN |


nil

# 2.4 What's the most common complaint type?


This is a really easy question to answer! There's a `frequencies` function that we can use:

In [96]:
;; Python: 
;; complaints['Complaint Type'].value_counts()

(->> fixed-df
     (map (keyword "Complaint Type"))
     frequencies
     (sort-by val >)
     (take 20))
(pp/pp)

(["HEATING" 14200]
 ["GENERAL CONSTRUCTION" 7471]
 ["Street Light Condition" 7117]
 ["DOF Literature Request" 5797]
 ["PLUMBING" 5373]
 ["PAINT - PLASTER" 5149]
 ["Blocked Driveway" 4590]
 ["NONCONST" 3998]
 ["Street Condition" 3473]
 ["Illegal Parking" 3343]
 ["Noise" 3321]
 ["Traffic Signal Condition" 3145]
 ["Dirty Conditions" 2653]
 ["Water System" 2636]
 ["Noise - Commercial" 2578]
 ["ELECTRIC" 2350]
 ["Broken Muni Meter" 2070]
 ["Noise - Street/Sidewalk" 1928]
 ["Sanitation Condition" 1824]
 ["Rodent" 1632])


nil

But it gets better! We can plot them!

In [131]:
(defn datapt [[s n]] {:Index s, :Value n})
(defn bar-graph [vs]
 {:data {:values vs}, 
  :mark "bar"
  :encoding {:x {:field :Index
                 :type "nominal"
                 :sort false}
             :y {:field :Value
                 :type "quantitative"}}})

(->> fixed-df
     (map (keyword "Complaint Type"))
     frequencies
     (sort-by val >)
     (take 10)
     (map datapt)
     bar-graph
     oz/view!)